# Import 

In [1]:
%pylab inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import os
from glob import glob
import pandas as pd
import numpy as np
from pandas import set_option
set_option('display.max_rows', 30)
from pathlib import Path
from scipy import integrate
import Process_interim
import Imaging
from datetime import datetime,timedelta,date,time

from uncertainties import ufloat
from uncertainties import umath
from uncertainties.umath import *

Populating the interactive namespace from numpy and matplotlib


/home/gus/anaconda3/lib/python3.8/site-packages/colour/utilities/verbose.py:237: ColourUsageWarning: "colour.models.XYZ_to_colourspace_model" object is deprecated and will be removed in a future release.
  warn(*args, **kwargs)


In [2]:
%watermark -a 'Gauthier Patin' -d -t -v -p numpy

Author: Gauthier Patin

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

numpy: 1.19.2



In [3]:
project = "3.4.9"

# Light values 

## Illuminance

In [4]:
Process_interim.Elsec_ill(project)

Data to be processed !


## Irradiance

In [5]:
Process_interim.Logbox_irr(project)

# RS 

In [62]:
cd/home/gus/Documents/PhD/PhD_Projects/Part3_MFT_Application/3.4_MFT-external/3.4.9_Inktime_Frank/Data/Interim/RS/To_process

/home/gus/Documents/PhD/PhD_Projects/Part3_MFT_Application/3.4_MFT-external/3.4.9_Inktime_Frank/Data/Interim/RS/To_process


In [63]:
# we import all the txt file
path = os.getcwd()
files_path = os.listdir(path)
files = [f for f in files_path if f[-3:] == 'txt']

dir_path = '/home/gus/Documents/PhD/PhD_Projects/Part3_MFT_Application/3.4_MFT-external/3.4.9_Inktime_Frank/Data/Processed/RS/'


# run the script to export the rawdata and process them
Process_interim.RS(files,dir_path,project, light_level='init')

ValueError: cannot set a row with mismatched columns

In [30]:
file = glob('**')[0]

df = pd.read_csv(file, index_col='parameter')
sp = pd.to_numeric(df['380':]['value1'])

In [32]:
files

Index(['380', '390', '400', '410', '420', '430', '440', '450', '460', '470',
       '480', '490', '500', '510', '520', '530', '540', '550', '560', '570',
       '580', '590', '600', '610', '620', '630', '640', '650', '660', '670',
       '680', '690', '700', '710', '720', '730', '740'],
      dtype='object', name='parameter')

In [25]:
(df.loc['wavelength_nm']['value1'])

'reflectance'

In [29]:
df['380':]

,value1,value2
parameter,,
380,0.0,NaN
390,0.0,NaN
400,0.23370000000000002,NaN
410,0.257,NaN
420,0.2699,NaN
...,...,...
700,0.4115,NaN
710,0.5015999999999999,NaN
720,0.5917,NaN


# AIS 

In [9]:
cd /home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/AIS/To_process

/home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/AIS/To_process


In [10]:
files = sorted(glob('*'))

# run the script to export the rawdata and process them
Process_interim.AIS_mean(files, project)

# Images 

In [13]:
cd /home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/Photos/To_process

/home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/Photos/To_process


## Flatfield 

In [14]:
files = glob('*TIF*')
im_white = glob('*background*')
im_obj = list(set(files)-set(im_white))

Imaging.flat_fielding(im_obj,im_white[0])

## ICC profile 

In [50]:
cd /home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/Photos

/home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/Photos


In [59]:
import pyvips
from pathlib import Path

def icc_transform(files,dir_path):
    date = files[0].split('_')[1]    
    
    output_profile = '/home/gus/Documents/PhD/PhD_Projects/Docs/profiles/eci_RGBv2.icc'
        
    input_profile_file = glob(f'*{date}*icc*')[0]    
    input_profile = Path(input_profile_file).resolve()
    input_profile = str(input_profile)

    for im_file in files:
        im = pyvips.Image.new_from_file('To_process/' + im_file, access = 'sequential')
        im_lab = im.icc_transform(output_profile, embedded=False, input_profile=input_profile, depth=8)
        
        #if im_lab.has_alpha:
        #    im_lab = im_lab.flatten(background = [128, 255, 128])
        
        filename = im_file.replace('flatfield','corr')
        im_lab.write_to_file(dir_path+filename)
        
        os.remove('To_process/' + im_file)

In [ ]:
files = os.listdir('/home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Interim/Photos/To_process')
dir_path = '/home/gus/Documents/PhD/PhD_Projects/Part2_Paint_&_colour_technology/2.6_Window-exp/Data/Processed/Photos/'

icc_transform(files,dir_path)